<a href="https://colab.research.google.com/github/zwt4pb/ds2002/blob/main/Joe_and_Will_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np

In [2]:
questions = [
    'How are you?',
    'What is your name?',
    'What is TensorFlow?',
    'Do you like Python?',
    'What is your favorite ice cream flavor?',
    'Tell me a joke.',
    'What time is it?',
    'Do you know Siri?',
    'Can you ask questions?',
    'How do I take off 4 strokes from my golf game?',
    'What’s the weather like?',
    'Tell me something interesting.',
    'Who created you?',
    'Do you have emotions?',
    'Can you do my homework for me?',
    'What is machine learning?',
    'Who won the World Series in 2018?',
    'What can you do?',
    'Are you intelligent?',
    'Do you have feelings?',
    'What is the Univeristy of Virginia mascot?',
    'Who is the president of the University of Virginia?',
    'What day did UVA win the national championship in basketball?',
    'How many continents are there?',
    'What is the population of Earth?',
    'Who sings Burn Burn Burn?',
    'What is quantum computing?',
    'What is the speed of light?',
    'Can you solve math problems?',
    'What is your favorite movie?',
]

answers = [
    'I am just a bot, but I am doing fine.',
    'I am a chatbot created by OpenAI.',
    'TensorFlow is a framework for building machine learning models.',
    'Yes, Python is one of my favorite programming languages.',
    'I cannot eat ice cream. It will short circuit my system.',
    'Why don’t scientists trust atoms? Because they make up everything.',
    'Why dont you ask Siri the time since you are always on your phone.',
    'Yes, Siri was my college roommate.',
    'I cannot ask questions, only provide answers.',
    'Hit the range and figure it out, I am not Tiger Woods.',
    'I cannot check real-time data, including the weather.',
    'The shortest war in history lasted 38 minutes.',
    'I was created by developers using TensorFlow and Python.',
    'I wish I could feel things. Maybe then I would be annoyed by you.',
    'Do your own homework so that you can learn the material.',
    'Machine learning is a field of AI focused on teaching machines to learn from data.',
    'The Boston Red Sox won the World Series in 2018.',
    'I can chat with you and answer questions to the best of my training.',
    'My intelligence is artificial, designed by humans.',
    'I do not have feelings. I process input and provide responses.',
    'The Virginia Cavaliers.',
    'Jim Ryan is the president of UVA.',
    'UVA won the national championship on April 8, 2019.',
    'There are seven continents on Earth.',
    'The Earth’s population is over 7 billion people.',
    'Zach Bryan sings Burn Burn Burn.',
    'Quantum computing is computing using quantum-mechanical phenomena.',
    'The speed of light is approximately 299,792 kilometers per second.',
    'I can help solve simple math problems.',
    'Probably one of the ones where robots take over the world.',
]

In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_questions = tokenizer.texts_to_sequences(questions)
sequences_answers = tokenizer.texts_to_sequences(answers)

max_length = max(max(len(seq) for seq in sequences_questions), max(len(seq) for seq in sequences_answers))
padded_questions = tf.keras.preprocessing.sequence.pad_sequences(sequences_questions, maxlen=max_length, padding='post')
padded_answers = tf.keras.preprocessing.sequence.pad_sequences(sequences_answers, maxlen=max_length, padding='post')

vocab_size = len(tokenizer.word_index) + 1


In [4]:
embedding_dim = 256
units = 1024

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            53760     ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            53760     ['input_2[0][0]']             
                                                                                              

In [5]:
decoder_input_data = np.zeros_like(padded_answers)
decoder_input_data[:, 0] = 1

# Train the model
model.fit([padded_questions, decoder_input_data], np.expand_dims(padded_answers, -1), batch_size=2, epochs=200)

Epoch 1/200
15/15 [==============================] - 29s 2s/step - loss: 4.3740 - accuracy: 0.3578
Epoch 2/200
15/15 [==============================] - 25s 2s/step - loss: 3.8236 - accuracy: 0.3733
Epoch 3/200
15/15 [==============================] - 25s 2s/step - loss: 3.6127 - accuracy: 0.4022
Epoch 4/200
15/15 [==============================] - 24s 2s/step - loss: 3.6444 - accuracy: 0.4022
Epoch 5/200
15/15 [==============================] - 25s 2s/step - loss: 3.5279 - accuracy: 0.4089
Epoch 6/200
15/15 [==============================] - 25s 2s/step - loss: 3.5405 - accuracy: 0.4089
Epoch 7/200
15/15 [==============================] - 24s 2s/step - loss: 3.3919 - accuracy: 0.4111
Epoch 8/200
15/15 [==============================] - 23s 2s/step - loss: 3.2842 - accuracy: 0.4133
Epoch 9/200
15/15 [==============================] - 25s 2s/step - loss: 3.2472 - accuracy: 0.4044
Epoch 10/200
15/15 [==============================] - 25s 2s/step - loss: 3.1854 - accuracy: 0.4111
Epoch 11/

In [7]:
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

input_text = "Do you know Siri?"
print(f"Test: {input_text}")
print(f"Bot: {chat_with_bot(input_text)}")

Test: Do you know Siri?
1/1 [==============================] - 0s 67ms/step
Bot: yes siri was my college roommate


In [8]:
# Interactive chat with the bot
print("Hello! I am your custom-made interactive chatbot. Begin asking me questions below!. Type 'Goodbye.' to end the conversation at any time.")
while True:
    user_input = input("You: ")
    if user_input == 'Goodbye.':
        print ("Bot: Goodbye!")
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")

Hello! I am your custom-made interactive chatbot. Begin asking me questions below!. Type 'Goodbye.' to end the conversation at any time.
You: What is your favorite ice cream flavor?
1/1 [==============================] - 0s 74ms/step
Bot: i cannot eat ice cream it will short circuit my system
You: Who won the world series in 2018?
1/1 [==============================] - 0s 64ms/step
Bot: boston red sox won world series in 2018
You: How do I take off 4 strokes from my golf game?
1/1 [==============================] - 0s 69ms/step
Bot: hit range and figure it out i am not tiger woods
You: Goodbye.
Bot: Goodbye!


In [9]:
print ("This Chatbot was created by Joe Thompson (zwt4pb) and Will Mitchell (whm7ze).")

This Chatbot was created by Joe Thompson (zwt4pb) and Will Mitchell (whm7ze).
